# Optimal Bidding
Statistical analysis of dataset and training model.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [2]:
def data_frame(string):
    df = pd.read_csv(string,names=[i for i in range(1,25)])
    lis = [i for i in range(len(df)+1)]
    df[0] =  pd.DataFrame({'col':lis})
    df.set_index(0)
    # print(df[2][4])
    #(df.head())
    l = []
    for i in range(len(df)):
        for x in range(1,25):
            l.insert(-1,df[x][i])
    df_new = pd.DataFrame()
    df_new['d_train'] = pd.DataFrame({'d_train':l})
#     print(df_new['d_train'].head())
    return df_new['d_train']

In [3]:
df_x = pd.DataFrame()
df_y = pd.DataFrame()

df_x['d_train'] = data_frame('Demand_Train.csv')
df_y['d_pred'] = data_frame('Demand_Train_pred.csv')

df_x['p_train'] = data_frame('Price_Train.csv')
df_y['p_pred'] = data_frame('Price_Train_pred.csv')

df_x['s_train'] = data_frame('Solar_Train.csv')
df_y['s_pred'] = data_frame('Solar_Train_pred.csv')
# df_new['PCT_change'] = (df[d_train])

In [4]:
df_x.fillna(-9999999,inplace=True)
df_y.fillna(-9999999,inplace=True)

In [5]:
print(df_x.head(),df_y.head())

(   d_train  p_train  s_train
0   151.40     1.60      0.0
1   154.94     1.40      0.0
2   152.50     1.37      0.0
3   148.54     1.36      0.0
4   148.57     2.80      0.0,    d_pred  p_pred  s_pred
0  132.99    1.56     0.0
1  160.67    1.31     0.0
2  154.47    1.50     0.0
3  126.27    1.33     0.0
4  158.16    2.92     0.0)


In [6]:
# print(df_x.head())
# # fig = plt.figure()
# # ax1 = plt.subplot2grid((1,1),(0,0))
# # ax2 = plt.subplot2grid((1,1),(1,1))
# x = (df_x['d_train']-df_x['s_train']) - (df_y['d_pred']-df_y['s_pred'])
# x.plot()
# # df_new['d_train'].plot(ax = ax2,label='Time')

# plt.legend()
# plt.show()

In [2]:
from sklearn import cross_validation, preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [8]:
X = np.array(df_x)
X = preprocessing.scale(X)
y = np.array(df_y).astype('float')
print(X[:5],y[:5])
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=.02)

(array([[ 1.31121183, -1.09962368, -0.77802507],
       [ 1.37310074, -1.26789807, -0.77802507],
       [ 1.33044285, -1.29313923, -0.77802507],
       [ 1.26121119, -1.30155295, -0.77802507],
       [ 1.26173567, -0.08997733, -0.77802507]]), array([[ 132.99,    1.56,    0.  ],
       [ 160.67,    1.31,    0.  ],
       [ 154.47,    1.5 ,    0.  ],
       [ 126.27,    1.33,    0.  ],
       [ 158.16,    2.92,    0.  ]]))


In [9]:
clf = LinearRegression()
# clf = RandomForestClassifier()
clf.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [10]:
clf.score(X_train,y_train)

0.98010230774102736

In [11]:
df_test = pd.DataFrame()
df_test['d'] = data_frame('Demand_LB_pred.csv')
df_test['p'] = data_frame('Price_LB_pred.csv')
df_test['s'] = data_frame('Solar_LB_pred.csv')

In [12]:
print(df_test.head())

       d     p    s
0  29.86  1.79  0.0
1  25.67  1.81  0.0
2  68.61  1.63  0.0
3  19.16  1.88  0.0
4  21.07  1.90  0.0


In [13]:
df_out = pd.DataFrame()
df_out = clf.predict(preprocessing.scale(df_test))

In [14]:
print(df_out[:5])

[[ 27.06075161   1.7912787    0.4463511 ]
 [ 22.35714427   1.81070136   0.44576045]
 [ 70.56070649   1.63589068   0.45172414]
 [ 15.04916286   1.87867009   0.44470341]
 [ 17.19329678   1.89808484   0.44486842]]


In [15]:
df_p = pd.DataFrame(df_out)
df_p.head()

,0,1,2
0,27.060752,1.791279,0.446351
1,22.357144,1.810701,0.445760
2,70.560706,1.635891,0.451724
3,15.049163,1.878670,0.444703
4,17.193297,1.898085,0.444868


In [16]:
df_p.describe()

,0,1,2
count,1200.000000,1200.000000,1200.000000
mean,76.396514,2.907314,6.863732
std,57.244774,1.184607,8.831745
min,10.053575,0.470922,0.435623
25%,25.402588,2.024273,0.447325
50%,64.105374,2.757987,0.461368
75%,110.720905,3.598076,13.427550
max,324.791809,8.093687,31.093252


In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(df_test['d'].astype('int'),df_p[0].astype('int'))

0.050833333333333335

In [33]:
d = pd.DataFrame()
d[1] = df_p[0] - df_p[2]
d[0] = df_p[1] + 0.2

In [34]:
for i in range(50):
    z = 0.
    for j in range(24):
        need = df_p[0][i*24+j]- df_p[2][i*24+j] 
        z = z + need * df_p[1][i*24+j]
    z = z/24.
    for j in range(24):
        need = df_p[0][i*24+j]- df_p[2][i*24+j]
        if need * df_p[1][i*24+j] > z:
            need = need - 4
        else:
            need = need + 5
        d[0][i*24+j] = need

In [35]:
d[1] = d[1].astype('int')

In [36]:
d[1]

0        26
1        21
2        70
3        14
4        16
5        46
6       140
7        71
8         2
9        -4
10       72
11      129
12       77
13       86
14       52
15       92
16       14
17       34
18       78
19       51
20      157
21       32
22       54
23       15
24       56
25       13
26       55
27       19
28       24
29       45
       ... 
1170    108
1171    171
1172    113
1173     23
1174     19
1175     15
1176     10
1177     12
1178     12
1179     17
1180     59
1181     61
1182    179
1183     92
1184     64
1185     16
1186     49
1187      4
1188     -8
1189     -4
1190      4
1191     21
1192     98
1193    130
1194    156
1195    103
1196     88
1197     80
1198     62
1199     28
Name: 1, Length: 1200, dtype: int64

In [37]:
d.reset_index()
d.to_csv('23.csv',index=False,header=None)